In [4]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from smodels.tools.databaseBrowser import Browser
import numpy as np

In [5]:
db = Browser('/home/lessa/smodels-database')

In [9]:
db.selectExpResultsWith(dataType='efficiencyMap',source='Fastlim-v1.0')

In [21]:
for exp in db:
    for dt in exp.datasets:
        for tx in dt.txnameList:            
            txdata = tx.txnameData
            if len(txdata.xsec) == 0:
                continue
            if max(txdata.xsec) == min(txdata.xsec) == 0.:
                print exp.globalInfo.id,dt.dataInfo.dataId,tx

ATLAS-CONF-2013-024 SR1: MET > 200 T1
ATLAS-CONF-2013-024 SR1: MET > 200 T2
ATLAS-CONF-2013-024 SR1: MET > 200 TGQ
ATLAS-CONF-2013-024 SR2: MET > 300 T1
ATLAS-CONF-2013-024 SR2: MET > 300 T2
ATLAS-CONF-2013-024 SR2: MET > 300 TGQ
ATLAS-CONF-2013-024 SR3: MET > 350 T1
ATLAS-CONF-2013-024 SR3: MET > 350 T2
ATLAS-CONF-2013-024 SR3: MET > 350 TGQ
ATLAS-CONF-2013-035 SRnoZa T1
ATLAS-CONF-2013-035 SRnoZa T1bbbb
ATLAS-CONF-2013-035 SRnoZa T1bbbt
ATLAS-CONF-2013-035 SRnoZa T1bbqq
ATLAS-CONF-2013-035 SRnoZa T1bbtt
ATLAS-CONF-2013-035 SRnoZa T1btbt
ATLAS-CONF-2013-035 SRnoZa T1btqq
ATLAS-CONF-2013-035 SRnoZa T1bttt
ATLAS-CONF-2013-035 SRnoZa T1qqtt
ATLAS-CONF-2013-035 SRnoZa T1tttt
ATLAS-CONF-2013-035 SRnoZa T2
ATLAS-CONF-2013-035 SRnoZa T2bb
ATLAS-CONF-2013-035 SRnoZa T2bt
ATLAS-CONF-2013-035 SRnoZa T2tt
ATLAS-CONF-2013-035 SRnoZa T5bbbb
ATLAS-CONF-2013-035 SRnoZa T5bbbt
ATLAS-CONF-2013-035 SRnoZa T5btbt
ATLAS-CONF-2013-035 SRnoZa T5tbtb
ATLAS-CONF-2013-035 SRnoZa T5tbtt
ATLAS-CONF-2013-035 SRn

In [6]:
m = [[300.*GeV, 257.158051402*GeV,250.059808177*GeV]]*2
porig = tx.txnameData.flattenMassArray(m)
print 'porig=',porig
pt = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
print pt
pt = np.dot(pt,tx.txnameData._V)  ## rotate
print pt
pt = np.array(pt[:tx.txnameData.dimensionality])
print 'Pt=',pt,'val=',tx.txnameData.getValueFor(m)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ptm = [pt[0],pt[1],tx.txnameData.getValueFor(m).asNumber(pb)]
# ax.scatter([ptm[0]], [ptm[1]], [ptm[2]],c='r',marker='o')
xpts = []
ypts = []
zpts = []
for ipt in tx.txnameData.tri.simplices[tx.txnameData.tri.find_simplex(pt)]:
    xpts.append(tx.txnameData.tri.points[ipt][0])
    ypts.append(tx.txnameData.tri.points[ipt][1])
    if tx.txnameData.dimensionality == 3:
        zpts.append(tx.txnameData.tri.points[ipt][2])
    else:
        zpts.append(tx.txnameData.xsec[ipt])
    print tx.txnameData.tri.points[ipt],tx.txnameData.xsec[ipt]

porig= [300.0, 257.158051402, 250.059808177, 300.0, 257.158051402, 250.059808177]
[-185.15625, 9.033051402000012, 15.908022462714285, -185.15625, 9.033051402000012, 15.908022462714285]
[ -2.18573519e+02  -1.46302848e+02   7.48929095e+00   1.21340438e-13
  -1.01354689e-13  -1.95399252e-14]
Pt= [-218.57351916 -146.30284845    7.48929095] val= 3.82E+00 [pb]
[-157.38735939 -110.77944446    9.40322604] 1.0006
[-212.56013946 -157.52258746   -5.46428627] 8.4695
[-193.56698182 -192.72533604   -5.62693608] 50.651
[-219.61752371 -144.35576474    9.59652735] 2.9203


In [ ]:
for pt in tx.txnameData._data:
    print pt[0][0],pt[1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xpts, ypts, zpts,s=80,c='r',marker='o')
ax.scatter([pt[0]],[pt[1]],[pt[2]],s=80,c='b',marker='o')
ax.set_xlabel('x')
ax.set_ylabel('y')
if tx.txnameData.dimensionality == 3:
    ax.set_zlabel('UL')
else:
    ax.set_zlabel('z')

plt.show()

In [ ]:
uvw = [pt[:3]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e06
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
print 'v=',tx.txnameData.wts
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
print 'r=',ret
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)